In [251]:
import numpy as np 
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack  

In [252]:
data = pd.read_csv('C:\\Users\\20100\\Desktop\\graduation project\\data\\IKEA_SA_Furniture_Web_Scrapings_sss.csv')
data.head()

,item_id,name,category,price,old_price,sellable_online,link,other_colors,short_description,designer,depth,height,width
0,90420332,FREKVENS,Bar furniture,265.0,No old price,True,https://www.ikea.com/sa/en/p/frekvens-bar-tabl...,No,"Bar table, in/outdoor, 51x51 cm",Nicholai Wiig Hansen,NaN,99.0,51.0
1,368814,NORDVIKEN,Bar furniture,995.0,No old price,False,https://www.ikea.com/sa/en/p/nordviken-bar-tab...,No,"Bar table, 140x80 cm",Francis Cayouette,NaN,105.0,80.0
2,9333523,NORDVIKEN / NORDVIKEN,Bar furniture,2095.0,No old price,False,https://www.ikea.com/sa/en/p/nordviken-nordvik...,No,Bar table and 4 bar stools,Francis Cayouette,NaN,NaN,NaN
3,80155205,STIG,Bar furniture,69.0,No old price,True,https://www.ikea.com/sa/en/p/stig-bar-stool-wi...,Yes,"Bar stool with backrest, 74 cm",Henrik Preutz,50.0,100.0,60.0
4,30180504,NORBERG,Bar furniture,225.0,No old price,True,https://www.ikea.com/sa/en/p/norberg-wall-moun...,No,"Wall-mounted drop-leaf table, ...",Marcus Arvonen,60.0,43.0,74.0


In [253]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3694 entries, 0 to 3693
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   item_id            3694 non-null   int64  
 1   name               3694 non-null   object 
 2   category           3694 non-null   object 
 3   price              3694 non-null   float64
 4   old_price          3694 non-null   object 
 5   sellable_online    3694 non-null   bool   
 6   link               3694 non-null   object 
 7   other_colors       3694 non-null   object 
 8   short_description  3694 non-null   object 
 9   designer           3694 non-null   object 
 10  depth              2231 non-null   float64
 11  height             2706 non-null   float64
 12  width              3105 non-null   float64
dtypes: bool(1), float64(4), int64(1), object(7)
memory usage: 350.0+ KB


In [254]:
df = data.copy()

In [255]:
df.duplicated().count()

3694

In [256]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1899 entries, 3 to 3688
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   item_id            1899 non-null   int64  
 1   name               1899 non-null   object 
 2   category           1899 non-null   object 
 3   price              1899 non-null   float64
 4   old_price          1899 non-null   object 
 5   sellable_online    1899 non-null   bool   
 6   link               1899 non-null   object 
 7   other_colors       1899 non-null   object 
 8   short_description  1899 non-null   object 
 9   designer           1899 non-null   object 
 10  depth              1899 non-null   float64
 11  height             1899 non-null   float64
 12  width              1899 non-null   float64
dtypes: bool(1), float64(4), int64(1), object(7)
memory usage: 194.7+ KB


In [257]:
df.describe()

,item_id,price,depth,height,width
count,1.899000e+03,1899.000000,1899.000000,1899.000000,1899.000000
mean,4.880518e+07,1220.626962,56.156398,113.109531,119.751975
std,2.915014e+07,1422.088876,30.579497,62.748882,77.515205
min,1.165950e+05,6.000000,1.000000,2.000000,2.000000
25%,2.034380e+07,295.000000,40.000000,71.000000,60.000000
50%,4.927859e+07,680.000000,47.000000,92.000000,93.000000
75%,7.043965e+07,1589.000000,60.000000,171.000000,161.500000
max,9.990379e+07,9585.000000,257.000000,301.000000,420.000000


In [258]:
df.head()

,item_id,name,category,price,old_price,sellable_online,link,other_colors,short_description,designer,depth,height,width
3,80155205,STIG,Bar furniture,69.0,No old price,True,https://www.ikea.com/sa/en/p/stig-bar-stool-wi...,Yes,"Bar stool with backrest, 74 cm",Henrik Preutz,50.0,100.0,60.0
4,30180504,NORBERG,Bar furniture,225.0,No old price,True,https://www.ikea.com/sa/en/p/norberg-wall-moun...,No,"Wall-mounted drop-leaf table, ...",Marcus Arvonen,60.0,43.0,74.0
5,10122647,INGOLF,Bar furniture,345.0,No old price,True,https://www.ikea.com/sa/en/p/ingolf-bar-stool-...,No,"Bar stool with backrest, 63 cm",Carina Bengs,45.0,91.0,40.0
6,70404875,FRANKLIN,Bar furniture,129.0,No old price,True,https://www.ikea.com/sa/en/p/franklin-bar-stoo...,No,"Bar stool with backrest, foldable, ...",K Hagberg/M Hagberg,44.0,95.0,50.0
8,50406465,FRANKLIN,Bar furniture,129.0,No old price,True,https://www.ikea.com/sa/en/p/franklin-bar-stoo...,No,"Bar stool with backrest, foldable, ...",K Hagberg/M Hagberg,44.0,95.0,50.0


In [259]:
df["category"].value_counts()

Bookcases & shelving units              375
Sofas & armchairs                       223
Chairs                                  213
Wardrobes                               199
Cabinets & cupboards                    199
Chests of drawers & drawer units        121
Tables & desks                          103
TV & media furniture                     95
Outdoor furniture                        81
Children's furniture                     78
Beds                                     78
Nursery furniture                        62
Bar furniture                            24
Sideboards, buffets & console tables     20
Café furniture                           13
Trolleys                                  9
Room dividers                             6
Name: category, dtype: int64

In [260]:
df["price"].dtype

dtype('float64')

In [261]:
df["old_price"] = df["old_price"].replace("No old price","").str.extract(r'^SR\s(.*)', expand=False)

In [262]:
df.head()

,item_id,name,category,price,old_price,sellable_online,link,other_colors,short_description,designer,depth,height,width
3,80155205,STIG,Bar furniture,69.0,NaN,True,https://www.ikea.com/sa/en/p/stig-bar-stool-wi...,Yes,"Bar stool with backrest, 74 cm",Henrik Preutz,50.0,100.0,60.0
4,30180504,NORBERG,Bar furniture,225.0,NaN,True,https://www.ikea.com/sa/en/p/norberg-wall-moun...,No,"Wall-mounted drop-leaf table, ...",Marcus Arvonen,60.0,43.0,74.0
5,10122647,INGOLF,Bar furniture,345.0,NaN,True,https://www.ikea.com/sa/en/p/ingolf-bar-stool-...,No,"Bar stool with backrest, 63 cm",Carina Bengs,45.0,91.0,40.0
6,70404875,FRANKLIN,Bar furniture,129.0,NaN,True,https://www.ikea.com/sa/en/p/franklin-bar-stoo...,No,"Bar stool with backrest, foldable, ...",K Hagberg/M Hagberg,44.0,95.0,50.0
8,50406465,FRANKLIN,Bar furniture,129.0,NaN,True,https://www.ikea.com/sa/en/p/franklin-bar-stoo...,No,"Bar stool with backrest, foldable, ...",K Hagberg/M Hagberg,44.0,95.0,50.0


In [263]:
df["sellable_online"].value_counts()

True     1886
False      13
Name: sellable_online, dtype: int64

In [264]:
# drop false rows 
df = df[df["sellable_online"] != False]

In [265]:
df["sellable_online"].value_counts()

True    1886
Name: sellable_online, dtype: int64

In [266]:
#df = df.drop(columns=["link"],axis=1)

In [267]:
df["other_colors"].value_counts()

No     979
Yes    907
Name: other_colors, dtype: int64

In [268]:
df.head()

,item_id,name,category,price,old_price,sellable_online,link,other_colors,short_description,designer,depth,height,width
3,80155205,STIG,Bar furniture,69.0,NaN,True,https://www.ikea.com/sa/en/p/stig-bar-stool-wi...,Yes,"Bar stool with backrest, 74 cm",Henrik Preutz,50.0,100.0,60.0
4,30180504,NORBERG,Bar furniture,225.0,NaN,True,https://www.ikea.com/sa/en/p/norberg-wall-moun...,No,"Wall-mounted drop-leaf table, ...",Marcus Arvonen,60.0,43.0,74.0
5,10122647,INGOLF,Bar furniture,345.0,NaN,True,https://www.ikea.com/sa/en/p/ingolf-bar-stool-...,No,"Bar stool with backrest, 63 cm",Carina Bengs,45.0,91.0,40.0
6,70404875,FRANKLIN,Bar furniture,129.0,NaN,True,https://www.ikea.com/sa/en/p/franklin-bar-stoo...,No,"Bar stool with backrest, foldable, ...",K Hagberg/M Hagberg,44.0,95.0,50.0
8,50406465,FRANKLIN,Bar furniture,129.0,NaN,True,https://www.ikea.com/sa/en/p/franklin-bar-stoo...,No,"Bar stool with backrest, foldable, ...",K Hagberg/M Hagberg,44.0,95.0,50.0


In [269]:
df = df.drop(columns=["designer"],axis=1)

In [270]:
df.to_csv("C:\\Users\\20100\\Desktop\\graduation project\\data\\furniture_store.csv")

In [271]:
df = pd.read_csv("C:\\Users\\20100\\Desktop\\graduation project\\ML Model\\furniture_store - Copy_db.csv",
                 usecols=["product_id","product_category","product_name","product_description"])
# df = pd.read_csv("C:\\Users\\20100\\Desktop\\graduation project\\ML Model\\roveconcepts2.csv",
#                  usecols=["product_category","product_name","product_description"])

In [272]:
df.head()

,product_id,product_category,product_name,product_description
0,30180504,Bar furniture,NORBERG,"Wall-mounted drop-leaf table, ..."
1,10122647,Bar furniture,INGOLF,"Bar stool with backrest, 63 cm"
2,70404875,Bar furniture,FRANKLIN,"Bar stool with backrest, foldable, ..."
3,50406465,Bar furniture,FRANKLIN,"Bar stool with backrest, foldable, ..."
4,90404879,Bar furniture,FRANKLIN,"Bar stool with backrest, foldable, ..."


In [273]:
if '30180504' in df['product_id'].astype(str).values:
    print("Value found in DataFrame.")
else:
    print("Value not found in DataFrame.")

Value found in DataFrame.


In [274]:
df = df.set_index('product_id')

In [275]:
df["product_name"].value_counts()

PAX                  101
BESTÅ                 96
EKET                  54
PLATSA                53
GRÖNLID               52
                    ... 
TERJE                  1
EKENÄS                 1
SVENARNE               1
FANBYN                 1
PAX / MEHAMN/AULI      1
Name: product_name, Length: 288, dtype: int64

### some of products name is redundant in many categories ,so it is not good to recommend by 'products_name'             ,So it is better to use product_id 

In [276]:
product_descriptions = df['product_description']
product_categories = df['product_category']
product_names = df['product_name']

In [277]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
count_vectorizer = CountVectorizer(stop_words='english')

In [278]:
tfidf_matrix = tfidf_vectorizer.fit_transform(product_descriptions)

category_matrix_weight = 2  
count_matrix_category = count_vectorizer.fit_transform(category_matrix_weight * product_categories)
count_matrix_name = count_vectorizer.fit_transform(product_names)
count_matrix = hstack([count_matrix_category, count_matrix_name])

In [279]:
# Combining the 2 matrices
combined_similarity_matrix = cosine_similarity(hstack([tfidf_matrix, count_matrix]))

In [280]:
def get_related_products(product_id, num_products=25):
    # Find the index of the input product
    if product_id not in df.index:
        print(f"{product_id} not found in product list.")
        return None
    idx = df.index.get_loc(product_id)

    # Get the cosine similarity scores of all products to the input product
    sim_scores = list(enumerate(combined_similarity_matrix[idx]))

    # Sort the products by their similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top num_products similar products
    sim_indices = [i[0] for i in sim_scores[1:num_products+1]]

    # Return the top num_products similar products
    return df.iloc[sim_indices][['product_category', 'product_name', 'product_description']]

In [281]:
related_products = get_related_products(10122647)
related_products

,product_category,product_name,product_description
product_id,,,
121766,Bar furniture,INGOLF,"Bar stool with backrest, 74 cm"
30352246,Bar furniture,RÅSKOG,"Bar stool, 63 cm"
30308395,Bar furniture,RÅSKOG,"Bar stool, 63 cm"
70404875,Bar furniture,FRANKLIN,"Bar stool with backrest, foldable, ..."
50406465,Bar furniture,FRANKLIN,"Bar stool with backrest, foldable, ..."
397736,Bar furniture,NORRARYD,"Bar stool with backrest, 74 cm"
90319918,Bar furniture,HENRIKSDAL,"Bar stool with backrest, 74 cm"
39184324,Bar furniture,HENRIKSDAL,"Bar stool with backrest, 74 cm"
20429010,Bar furniture,NORRÅKER,"Bar stool with backrest, 74 cm"


######  get_related_products
<ol>
    
<li>First, it checks if the input product_id exists in the index of the DataFrame df.
  If the product_id is not found, the function prints a message and returns None.
<li>If the product_id is found, the function gets the index idx of the input product in the DataFrame df.
<li>The function then gets the cosine similarity scores between the input product and all other products in the DataFrame df.
  It does this by getting the idx-th row of the combined_similarity_matrix.
<li>The similarity scores are then sorted in descending order, so that the most similar products are first.
<li>The function then selects the top num_products similar products (excluding the input product itself)
  by getting the indices of the products with the highest similarity scores. 
    These indices are stored in the sim_indices variable.
<li>Finally, the function returns a DataFrame containing the product category, name, and description
  of the top num_products similar products, as indexed by sim_indices.
    </ol>

In [282]:
df = df.reset_index()
indices = pd.Series(df.index, index=df['product_id'])

def get_recommendations(id, cosine_sim=combined_similarity_matrix):
    # Get the index of the movie that matches the title
    idx = indices[id]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:25]

    # Get the movie indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['product_id'].iloc[product_indices]

In [283]:
get_recommendations(10122647)

5         121766
12      30352246
21      30308395
2       70404875
3       50406465
6         397736
16      90319918
17      39184324
18      20429010
8         400550
14        400545
15      10400540
7       50420329
9       40426138
19        272656
4       90404879
11      60406785
10      70246089
13      10281354
20      30135658
0       30180504
665       152282
1000    80208529
1026    70418170
Name: product_id, dtype: int64